In [1]:
import requests
import pandas as pd
import re
import time

In [2]:
# NOAA NCEI API 需要一个 API Key
API_TOKEN = "XbyuERiTrXrJMbExiMRwzCNOZKwjQQep"  # 需要去 https://www.ncdc.noaa.gov/cdo-web/token 申请

HEADERS = {
    "token": API_TOKEN
}

def get_noaa_station_id(lat, lon):
    """ 通过经纬度查询 NOAA 气象站 ID """
    # 定义查询范围（latitude +/- 0.5, longitude +/- 0.5 以找到最近站点）
    min_lat, max_lat = lat - 0.1, lat + 0.1
    min_lon, max_lon = lon - 0.1, lon + 0.1
    
    # 构造 API 请求 URL
    stations_url = f"https://www.ncdc.noaa.gov/cdo-web/api/v2/stations"
    params = {
        "extent": f"{min_lat},{min_lon},{max_lat},{max_lon}",  # 指定查询范围
        "datasetid": "GHCND",  # 选择 GHCND 数据集
        "limit": 5  # 限制返回站点数
    }
    response = requests.get(stations_url, headers=HEADERS, params=params)

    if response.status_code == 200:
        data = response.json()
        if "results" in data:
            stations = data["results"]
            print('Succeeded!')
            return stations[0]['id']  # 返回最近的气象站 ID
        else:
            print("没有找到合适的气象站")
            return None
    else:
        print("请求 NOAA API 失败:", response.status_code, response.text)
        return None

In [15]:
country = 'USA-NPN_individual_phenometrics_data'
df = pd.read_csv(f'{country}.csv')
df.rename(columns={'State': 'location', 'Latitude': 'lat', 'Longitude': 'long'}, inplace=True)
# df['location'] = df['location'].str.replace('South Korea/', '')
df['location'] = df['location'].str.lower()

In [21]:
df = df[['location', 'lat', 'long']].copy()
df.drop_duplicates(inplace=True)
df.head()

,location,lat,long
0,nc,35.926178,-78.692719
2,nc,35.875225,-78.737076
3,dc,38.886761,-77.041016
6,tn,35.879211,-84.172203
7,ca,37.799515,-122.412834


In [27]:
df.shape

(89, 3)

In [61]:
station_list = []

In [ ]:
for index, row in df.iterrows():
    if row['lat'] in station_dict:
        print(f'The ID of {row['lat']} has already been obtained.')
    else:
        # 目标经纬度
        latitude = row['lat'] 
        longitude = row['long']
        print(f'Obtaining {row['location']}\'s station ID....')
        # 获取气象站 ID
        station = get_noaa_station_id(latitude, longitude)
        if not station:
            print(f'The ID of {row['location']} is none.')
        else:
            station_dict = {}
            station_dict['lat'] = latitude
            station_dict['long'] = longitude
            station_dict['STATION'] = station.replace('GHCND:','')
            station_list.append(station_dict)
        time.sleep(2)

In [ ]:
for index, row in df.iterrows():
    if row['location'] in station_dict:
        print(f'The ID of {row['location']} has already been obtained.')
    else:
        # 目标经纬度
        latitude = row['lat'] 
        longitude = row['long']
        print(f'Obtaining {row['location']}\'s station ID....')
        # 获取气象站 ID
        station = get_noaa_station_id(latitude, longitude)
        if not station:
            print(f'The ID of {row['location']} is none.')
        else:
            station_dict[row['location']] = statistation_dfon.replace('GHCND:','')
        time.sleep(2)

In [43]:
station_df = pd.DataFrame([station_dict])

In [173]:
station_df = pd.DataFrame(list(station_dict.items()), columns=['location', 'STATION'])

In [177]:
station_df.to_csv('korea_station_id.csv', index=False)

In [175]:
station_df

,location,STATION
0,sokcho,KSW00043254
1,chuncheon,KSM00047101
2,gangneung,KSM00047105
3,seoul,KSM00047108
4,incheon,KS000047112
5,wonju,KSW00043222
6,ulleungdo,KS000047115
7,suwon,KSW00043206
8,daejeon,KSM00047133
9,pohang,KSM00047138
